In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib as plt
from pathlib import Path


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Rpabo\.conda\envs\rait_env\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Rpabo\.conda\envs\rait_env\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Rpabo\.conda\envs\rait_env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Rpabo\.conda\envs\rait_env\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
    app.start()
  

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [2]:
# Tuning Values

HPARAMS_DEFAULTS = 0
HPARAMS_PAPER = 1
HPARAMS_CUSTOM = 2
training_type = HPARAMS_CUSTOM


OP_RMSPROP = 0
OP_SGD = 1
op_type = OP_RMSPROP

if (training_type == HPARAMS_DEFAULTS):
    num_hidden_layers = 1
    num_hidden_layer_neurons = 32   # X
    dropout_rate = 0.4              # X
    batch_size = 32                 # X 
    optimizer_learning_rate = 0.01  # X  
    gradient_decay_rho = 0.9
    learning_rate_decay = 0.2       # X
    hidden_layer_activation_function = "relu"
    #-----
    op_type = OP_RMSPROP
    training_epochs = 30
elif (training_type == HPARAMS_PAPER):
    num_hidden_layers = 1
    num_hidden_layer_neurons = 256  # X
    dropout_rate = 0.4              # X
    batch_size = 2275               # X 
    optimizer_learning_rate = 3e-4  # X // 3e-4, or 3*(10^-4) = 0.0003
    gradient_decay_rho = 0.9
    learning_rate_decay = 0.2       # X
    hidden_layer_activation_function = "tanh"
    #-----
    op_type = OP_RMSPROP
    training_epochs = 30
elif (training_type == HPARAMS_CUSTOM):
    num_hidden_layers = 1
    num_hidden_layer_neurons = 135     # X
    dropout_rate = 0.2596709650832801  # X
    batch_size = 2275                  # X 
    optimizer_learning_rate = 0.0029044390072926946   # X // 3e-4, or 3*(10^-4) = 0.0003
    gradient_decay_rho = 0.7437864402038941
    learning_rate_decay = 0.39953909944182836         # X
    hidden_layer_activation_function = "tanh"
    #-----
    op_type = OP_RMSPROP
    training_epochs = 30

'''
Paper Results: 
Epoch 30/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7001 - loss: 0.5711 - val_accuracy: 0.9895 - val_loss: 0.1341

Custom Results:
Epoch 30/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8090 - loss: 0.4114 - val_accuracy: 0.9987 - val_loss: 0.0293


Trial 05 summary
Hyperparameters:
n_hidden: 1
n_neurons: 135
learning_rate: 0.0029044390072926946
dropout_rate: 0.2596709650832801
optimizer: rmsprop
rho: 0.7437864402038941
learning_rate_decay: 0.39953909944182836
decay_steps: 9028
Score: 1.0
'''

output_layer_activation_function = "sigmoid"
tf.keras.utils.set_random_seed(42)


use_gpu = True
if not use_gpu:
    import os
    #os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" #If the line below doesn't work, uncomment this line (make sure to comment the line below); it should help.
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
train_path = Path("../../feature_data/train/featureInput_noChampionRole.csv")
test_path = Path("../../feature_data/test/featureInput_noChampionRole.csv")

#names = ["Match Result", "Blue Top Player", "Blue Jungle Player", "Blue Middle Player", "Blue ADC Player", "Blue Support Player", "Red Top Player", "Red Jungle Player", "Red Middle Player", "Red ADC Player", "Red Support Player", "Blue Top Champion", "Blue Jungle Champion", "Blue Middle Champion", "Blue ADC Champion", "Blue Support Champion", "Red Top Champion", "Red Jungle Champion", "Red Middle Champion", "Red ADC Champion", "Red Support Champion", "Blue Player Cooperation", "Red Player Cooperation", "Blue vs Red Player", "Blue Champion Cooperation", "Red Champion Cooperation", "Blue vs Red Champion", "Blue Team Win Rate When Blue", "Red Team Win Rate When Red"]
names = ["bResult", "btPlayerRole", "bjPlayerRole", "bmPlayerRole", "baPlayerRole", "bsPlayerRole", "rtPlayerRole", "rjPlayerRole", "rmPlayerRole", "raPlayerRole", "rsPlayerRole", "btPlayerChampion", "bjPlayerChampion", "bmPlayerChampion", "baPlayerChampion", "bsPlayerChampion", "rtPlayerChampion", "rjPlayerChampion", "rmPlayerChampion", "raPlayerChampion", "rsPlayerChampion", "bCoopPlayer", "rCoopPlayer", "vsPlayer", "bCoopChampion", "rCoopChampion", "vsChampion", "bTeamColor", "rTeamColor"]

# Load our pre-processed training and test sets. 
# The skiprows value just removes the first row of the CSV which are labels instead of data.
train_data = (pd.read_csv(train_path, names=names, skiprows=1))
test_data  = (pd.read_csv( test_path, names=names, skiprows=1))

In [4]:
train_data.head()

,bResult,btPlayerRole,bjPlayerRole,bmPlayerRole,baPlayerRole,bsPlayerRole,rtPlayerRole,rjPlayerRole,rmPlayerRole,raPlayerRole,...,raPlayerChampion,rsPlayerChampion,bCoopPlayer,rCoopPlayer,vsPlayer,bCoopChampion,rCoopChampion,vsChampion,bTeamColor,rTeamColor
0,1,0.583333,0.523810,0.708029,0.613445,0.583333,0.483146,0.516667,0.476923,0.596899,...,0.521739,0.000000,13.767162,13.040247,13.580071,12.360696,13.734771,12.361474,0.708029,0.596899
1,0,0.560000,0.500000,0.583333,0.425926,0.461538,0.462963,0.500000,0.322581,0.500000,...,0.533333,0.555556,11.152535,9.888245,1.611111,10.746699,12.583699,11.321637,0.473684,0.400000
2,1,0.483871,0.700000,0.669643,0.511905,0.750000,0.630435,0.533333,0.504762,0.583333,...,0.625000,0.000000,12.236232,9.672135,19.809739,12.499868,12.036395,14.065184,0.600000,0.625000
3,0,0.571429,0.550000,0.538462,0.514019,0.633663,0.384615,0.562500,0.408163,1.000000,...,1.000000,0.578947,13.148958,19.120690,6.476123,12.645715,12.816541,12.902756,0.512821,0.381356
4,1,1.000000,0.651515,0.666667,0.739130,0.624060,0.350000,0.350000,0.300000,0.250000,...,0.000000,0.000000,14.449021,10.512264,25.000000,13.162254,11.364051,12.801331,0.610294,0.391304


In [5]:
len(train_data)

6823

In [6]:
len(test_data)

759

In [7]:
train_data.shape

(6823, 29)

In [8]:
test_data.shape

(759, 29)

In [9]:
train_data.dtypes

bResult               int64
btPlayerRole        float64
bjPlayerRole        float64
bmPlayerRole        float64
baPlayerRole        float64
bsPlayerRole        float64
rtPlayerRole        float64
rjPlayerRole        float64
rmPlayerRole        float64
raPlayerRole        float64
rsPlayerRole        float64
btPlayerChampion    float64
bjPlayerChampion    float64
bmPlayerChampion    float64
baPlayerChampion    float64
bsPlayerChampion    float64
rtPlayerChampion    float64
rjPlayerChampion    float64
rmPlayerChampion    float64
raPlayerChampion    float64
rsPlayerChampion    float64
bCoopPlayer         float64
rCoopPlayer         float64
vsPlayer            float64
bCoopChampion       float64
rCoopChampion       float64
vsChampion          float64
bTeamColor          float64
rTeamColor          float64
dtype: object

In [10]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28]))
model.add(tf.keras.layers.Dropout(rate=dropout_rate))
for _ in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_hidden_layer_neurons, activation=hidden_layer_activation_function))
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))
model.add(tf.keras.layers.Dense(1, activation=output_layer_activation_function))

if op_type == OP_RMSPROP:
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=optimizer_learning_rate, rho=gradient_decay_rho),
        metrics=["accuracy"])
else:
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.SGD(learning_rate=optimizer_learning_rate, decay=learning_rate_decay),
        metrics=["accuracy"])

In [11]:
y_train = train_data["bResult"]
y_test  =  test_data["bResult"]

In [12]:
# Remove the match result column from the data.
X_train = train_data.drop("bResult", axis=1)
X_test  =  test_data.drop("bResult", axis=1)

In [13]:
import time
start_time = time.time()

In [14]:
# Train the model.

history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=training_epochs,
    batch_size=batch_size)

Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.5428 - loss: 0.7661 - val_accuracy: 0.9565 - val_loss: 0.2857
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6518 - loss: 0.6438 - val_accuracy: 0.9723 - val_loss: 0.2137
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6777 - loss: 0.6044 - val_accuracy: 0.9710 - val_loss: 0.1793
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6994 - loss: 0.5810 - val_accuracy: 0.9605 - val_loss: 0.1890
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7034 - loss: 0.5790 - val_accuracy: 0.9776 - val_loss: 0.1524
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7215 - loss: 0.5491 - val_accuracy: 0.9776 - val_loss: 0.1422
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7383 - loss: 0.5241 - val_accuracy: 0.9816 - val_loss: 0.1348
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7303 - loss: 0.5295 - val_accuracy: 0.9802 - val_loss: 0.1269

In [15]:
train_time = time.time() - start_time
train_time

4.583531141281128

In [16]:
import keras_tuner as kt

def build_model(hp):
    tune_n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=1)
    tune_n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    tune_learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    tune_dropout_rate = hp.Float("dropout_rate", min_value=0.1, max_value=0.9)
    tune_optimizer = hp.Choice("optimizer", values=["sgd", "rmsprop"])
    
    tune_rho = hp.Float("rho", min_value=0.7, max_value=1.0)
    tune_learning_rate_decay = hp.Float("learning_rate_decay", min_value=0.01, max_value=0.99)
    tune_decay_steps = hp.Int("decay_steps", min_value=1000, max_value=100000)
    tune_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=tune_learning_rate,
        decay_steps=tune_decay_steps,
        decay_rate=tune_learning_rate_decay)
    
    if tune_optimizer == "sgd":
        tune_optimizer = tf.keras.optimizers.SGD(learning_rate=tune_lr_schedule)
    else:
        tune_optimizer = tf.keras.optimizers.RMSprop(learning_rate=tune_learning_rate, rho=tune_rho)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=[28]))
    model.add(tf.keras.layers.Dropout(rate=tune_dropout_rate))
    for _ in range(tune_n_hidden):
        model.add(tf.keras.layers.Dense(tune_n_neurons, activation=hidden_layer_activation_function))
        model.add(tf.keras.layers.Dropout(rate=tune_dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation=output_layer_activation_function))

    model.compile(
        loss="binary_crossentropy",
        optimizer=tune_optimizer,
        metrics=["accuracy"])

    return model

In [17]:
random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=10, overwrite=True, directory="results", project_name="tune_search", seed=42)
random_search_tuner.search(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=training_epochs,
    batch_size=batch_size
)

Trial 10 Complete [00h 00m 08s]
val_accuracy: 0.9802371263504028

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 01m 11s


In [18]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

C:\Users\scotty\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
C:\Users\scotty\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
C:\Users\scotty\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [19]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values

{'n_hidden': 1,
 'n_neurons': 135,
 'learning_rate': 0.0029044390072926946,
 'dropout_rate': 0.2596709650832801,
 'optimizer': 'rmsprop',
 'rho': 0.7437864402038941,
 'learning_rate_decay': 0.39953909944182836,
 'decay_steps': 9028}

In [20]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 05 summary
Hyperparameters:
n_hidden: 1
n_neurons: 135
learning_rate: 0.0029044390072926946
dropout_rate: 0.2596709650832801
optimizer: rmsprop
rho: 0.7437864402038941
learning_rate_decay: 0.39953909944182836
decay_steps: 9028
Score: 1.0


In [21]:
best_trial.metrics.get_last_value("val_accuracy")

np.float64(1.0)

In [22]:
#best_model.fit(X_train, y_train, epochs=10)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0448
